In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import scipy.stats as st
import datetime as dt
from pandas_datareader import data 
import yfinance
import sqlite3
# Supress Warnings
import warnings
warnings.simplefilter(action = "ignore")

In [2]:
# Obtain the closing prices of stock for the analysis
yfinance.pdr_override() 

# Extraction of Data

In [3]:
# Fetching the data from Yahoo Finance for Apple (AAPL)
# Creating the Price table for Apple for 2023 
apple_df = data.get_data_yahoo("AAPL", start = "2023-01-01", end = "2023-12-31")
apple_df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000,124.216301,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,125.497498,89113600
2023-01-05,127.129997,127.769997,124.760002,125.019997,124.166641,80962700
2023-01-06,126.010002,130.289993,124.889999,129.619995,128.735229,87754700
2023-01-09,130.470001,133.410004,129.889999,130.149994,129.261612,70790800


# Transforming Data

In [4]:
# Creating the Volume table for Apple based on the dataframe above
apple_volume = apple_df[["Adj Close", "Volume"]]
apple_volume["stock_id"] = "AAPL"

# Reset the index and update add relevant columns
apple_volume.reset_index(inplace = True)
apple_volume = apple_volume[["Date", "stock_id", "Adj Close", "Volume"]]
apple_volume.head()

,Date,stock_id,Adj Close,Volume
0,2023-01-03,AAPL,124.216301,112117500
1,2023-01-04,AAPL,125.497498,89113600
2,2023-01-05,AAPL,124.166641,80962700
3,2023-01-06,AAPL,128.735229,87754700
4,2023-01-09,AAPL,129.261612,70790800


In [5]:
# Dropping Adj Close and Volume columns from the price table
apple_df.drop(columns = ["Adj Close", "Volume"], inplace = True)
apple_df.head()

,Open,High,Low,Close
Date,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000
2023-01-04,126.889999,128.660004,125.080002,126.360001
2023-01-05,127.129997,127.769997,124.760002,125.019997
2023-01-06,126.010002,130.289993,124.889999,129.619995
2023-01-09,130.470001,133.410004,129.889999,130.149994


In [6]:
# Use Lambda function to round all numbers to two decimal places
apple_df = apple_df.apply(lambda x: round(x,2))

# Compute Percentage Change in Stock Price for Apple
apple_df["Percentage_Change"] = apple_df["Close"].pct_change()
apple_df.reset_index(inplace = True)

# Display the updated dataframe
apple_df.head()

,Date,Open,High,Low,Close,Percentage_Change
0,2023-01-03,130.28,130.90,124.17,125.07,NaN
1,2023-01-04,126.89,128.66,125.08,126.36,0.010314
2,2023-01-05,127.13,127.77,124.76,125.02,-0.010605
3,2023-01-06,126.01,130.29,124.89,129.62,0.036794
4,2023-01-09,130.47,133.41,129.89,130.15,0.004089


In [7]:
# Add a monthly enumerator based on the date column within the above dataframe
apple_df["Month"] = apple_df["Date"].dt.month

# Display the updated dataframe
apple_df.head()

,Date,Open,High,Low,Close,Percentage_Change,Month
0,2023-01-03,130.28,130.90,124.17,125.07,NaN,1
1,2023-01-04,126.89,128.66,125.08,126.36,0.010314,1
2,2023-01-05,127.13,127.77,124.76,125.02,-0.010605,1
3,2023-01-06,126.01,130.29,124.89,129.62,0.036794,1
4,2023-01-09,130.47,133.41,129.89,130.15,0.004089,1


In [8]:
# Normalize the stock price based on the closing price of the first calendar day of trading
apple_df["Normalization"] = apple_df["Close"]/125.07
apple_df["stock_id"] = "AAPL"

# Display the updated dataframe with the normalized factor for the price
apple_df.head()

,Date,Open,High,Low,Close,Percentage_Change,Month,Normalization,stock_id
0,2023-01-03,130.28,130.90,124.17,125.07,NaN,1,1.000000,AAPL
1,2023-01-04,126.89,128.66,125.08,126.36,0.010314,1,1.010314,AAPL
2,2023-01-05,127.13,127.77,124.76,125.02,-0.010605,1,0.999600,AAPL
3,2023-01-06,126.01,130.29,124.89,129.62,0.036794,1,1.036380,AAPL
4,2023-01-09,130.47,133.41,129.89,130.15,0.004089,1,1.040617,AAPL


In [9]:
# Calculate the cumulative product of the normalized price change and store in a new column
apple_df["Accumulation"] = (apple_df["Percentage_Change"]+1).cumprod()

# Display the updated dataframe
apple_df.head()

,Date,Open,High,Low,Close,Percentage_Change,Month,Normalization,stock_id,Accumulation
0,2023-01-03,130.28,130.90,124.17,125.07,NaN,1,1.000000,AAPL,NaN
1,2023-01-04,126.89,128.66,125.08,126.36,0.010314,1,1.010314,AAPL,1.010314
2,2023-01-05,127.13,127.77,124.76,125.02,-0.010605,1,0.999600,AAPL,0.999600
3,2023-01-06,126.01,130.29,124.89,129.62,0.036794,1,1.036380,AAPL,1.036380
4,2023-01-09,130.47,133.41,129.89,130.15,0.004089,1,1.040617,AAPL,1.040617


In [10]:
# Apply lambda function to round the relevant dataframe values to four decimal places
apple_df[[
    "Percentage_Change", 
    "Normalization", 
    "Accumulation"
]] = apple_df[[
    "Percentage_Change", 
    "Normalization", 
    "Accumulation"
]].apply(lambda x: round(x,4))

# Display the updated dataframe
apple_df.head()

,Date,Open,High,Low,Close,Percentage_Change,Month,Normalization,stock_id,Accumulation
0,2023-01-03,130.28,130.90,124.17,125.07,NaN,1,1.0000,AAPL,NaN
1,2023-01-04,126.89,128.66,125.08,126.36,0.0103,1,1.0103,AAPL,1.0103
2,2023-01-05,127.13,127.77,124.76,125.02,-0.0106,1,0.9996,AAPL,0.9996
3,2023-01-06,126.01,130.29,124.89,129.62,0.0368,1,1.0364,AAPL,1.0364
4,2023-01-09,130.47,133.41,129.89,130.15,0.0041,1,1.0406,AAPL,1.0406


In [11]:
# Rearrange the dataframe in the desired order of columns for further analysis and clarity
apple_df = apple_df[[
    "Date",
    "stock_id",
    "Open",
    "High",
    "Low",
    "Close",
    "Month",
    "Normalization",
    "Percentage_Change",
    "Accumulation"
]]
# Display the updated dataframe
apple_df.head()

,Date,stock_id,Open,High,Low,Close,Month,Normalization,Percentage_Change,Accumulation
0,2023-01-03,AAPL,130.28,130.90,124.17,125.07,1,1.0000,NaN,NaN
1,2023-01-04,AAPL,126.89,128.66,125.08,126.36,1,1.0103,0.0103,1.0103
2,2023-01-05,AAPL,127.13,127.77,124.76,125.02,1,0.9996,-0.0106,0.9996
3,2023-01-06,AAPL,126.01,130.29,124.89,129.62,1,1.0364,0.0368,1.0364
4,2023-01-09,AAPL,130.47,133.41,129.89,130.15,1,1.0406,0.0041,1.0406


In [12]:
# Define a generic function to read the stock data from Yahoo Finance and insert into a dataframe
def stocks_to_df(Ticker, start = "2023-01-01", end = "2023-12-31"):
    temp_df = data.get_data_yahoo(Ticker, start=start, end=end)
    temp_df = temp_df.apply(lambda x: round(x,2))
    temp_vol = temp_df[["Adj Close", "Volume"]]
    temp_vol["stock_id"] = Ticker
    temp_vol.reset_index(inplace=True)
    temp_vol = temp_vol[["Date","stock_id","Adj Close","Volume"]]
    temp_df.drop(columns= ["Adj Close", "Volume"], inplace=True)
    temp_df["Percentage_Change"] = temp_df["Close"].pct_change()
    temp_df.reset_index(inplace=True)
    temp_df["Month"] = temp_df["Date"].dt.month
    temp_df["Normalization"] = temp_df["Close"]/temp_df["Close"][0]
    temp_df["stock_id"] = Ticker
    temp_df["Accumulation"] = (temp_df["Percentage_Change"]+1).cumprod()
    temp_df[[
        "Normalization",
        "Percentage_Change",
        "Accumulation"
    ]]=temp_df[[
        "Normalization",
        "Percentage_Change",
        "Accumulation"
    ]].apply(lambda x:round(x,4))
    temp_df = temp_df[[
        "Date",
        "stock_id",
        "Open",
        "High",
        "Low",
        "Close",
        "Month",
        "Normalization",
        "Percentage_Change",
        "Accumulation"
    ]]

    return temp_df,temp_vol   

In [13]:
# Load all relevant data related to Google Stock based on the earlier defined function
google_df = stocks_to_df("GOOGL")
print("-----------Price Table-----------")
display(google_df[0].head())
print("-----------Volume Table-----------")
display(google_df[1].head())

[*********************100%%**********************]  1 of 1 completed

-----------Price Table-----------


,Date,stock_id,Open,High,Low,Close,Month,Normalization,Percentage_Change,Accumulation
0,2023-01-03,GOOGL,89.59,91.05,88.52,89.12,1,1.0000,NaN,NaN
1,2023-01-04,GOOGL,90.35,90.65,87.27,88.08,1,0.9883,-0.0117,0.9883
2,2023-01-05,GOOGL,87.47,87.57,85.90,86.20,1,0.9672,-0.0213,0.9672
3,2023-01-06,GOOGL,86.79,87.69,84.86,87.34,1,0.9800,0.0132,0.9800
4,2023-01-09,GOOGL,88.36,90.05,87.86,88.02,1,0.9877,0.0078,0.9877


-----------Volume Table-----------


,Date,stock_id,Adj Close,Volume
0,2023-01-03,GOOGL,89.12,28131200
1,2023-01-04,GOOGL,88.08,34854800
2,2023-01-05,GOOGL,86.20,27194400
3,2023-01-06,GOOGL,87.34,41381500
4,2023-01-09,GOOGL,88.02,29003900


In [14]:
# Load all relevant data related to Intel Stock based on the earlier defined function
intel_df = stocks_to_df("INTC")
print("-----------Price Table-----------")
display(intel_df[0].head())
print("-----------Volume Table-----------")
display(intel_df[1].head())

[*********************100%%**********************]  1 of 1 completed

-----------Price Table-----------


,Date,stock_id,Open,High,Low,Close,Month,Normalization,Percentage_Change,Accumulation
0,2023-01-03,INTC,27.05,27.07,26.37,26.73,1,1.0000,NaN,NaN
1,2023-01-04,INTC,27.41,27.92,27.16,27.68,1,1.0355,0.0355,1.0355
2,2023-01-05,INTC,27.44,27.78,27.38,27.56,1,1.0311,-0.0043,1.0311
3,2023-01-06,INTC,27.93,28.83,27.38,28.73,1,1.0748,0.0425,1.0748
4,2023-01-09,INTC,28.84,29.87,28.84,29.31,1,1.0965,0.0202,1.0965


-----------Volume Table-----------


,Date,stock_id,Adj Close,Volume
0,2023-01-03,INTC,26.04,31308000
1,2023-01-04,INTC,26.97,37665200
2,2023-01-05,INTC,26.85,33552600
3,2023-01-06,INTC,27.99,31788600
4,2023-01-09,INTC,28.56,37493200


In [15]:
# Load all relevant data related to Microsoft Stock based on the earlier defined function
microsoft_df = stocks_to_df("MSFT")
print("-----------Price Table-----------")
display(microsoft_df[0].head())
print("-----------Volume Table-----------")
display(microsoft_df[1].head())

[*********************100%%**********************]  1 of 1 completed

-----------Price Table-----------


,Date,stock_id,Open,High,Low,Close,Month,Normalization,Percentage_Change,Accumulation
0,2023-01-03,MSFT,243.08,245.75,237.40,239.58,1,1.0000,NaN,NaN
1,2023-01-04,MSFT,232.28,232.87,225.96,229.10,1,0.9563,-0.0437,0.9563
2,2023-01-05,MSFT,227.20,227.55,221.76,222.31,1,0.9279,-0.0296,0.9279
3,2023-01-06,MSFT,223.00,225.76,219.35,224.93,1,0.9389,0.0118,0.9389
4,2023-01-09,MSFT,226.45,231.24,226.41,227.12,1,0.9480,0.0097,0.9480


-----------Volume Table-----------


,Date,stock_id,Adj Close,Volume
0,2023-01-03,MSFT,237.04,25740000
1,2023-01-04,MSFT,226.67,50623400
2,2023-01-05,MSFT,219.95,39585600
3,2023-01-06,MSFT,222.54,43613600
4,2023-01-09,MSFT,224.71,27369800


In [16]:
# Load all relevant data related to Oracle Stock based on the earlier defined function
oracle_df = stocks_to_df("ORCL")
print("-----------Price Table-----------")
display(oracle_df[0].head())
print("-----------Volume Table-----------")
display(oracle_df[1].head())

[*********************100%%**********************]  1 of 1 completed

-----------Price Table-----------


,Date,stock_id,Open,High,Low,Close,Month,Normalization,Percentage_Change,Accumulation
0,2023-01-03,ORCL,82.47,83.84,82.26,83.72,1,1.0000,NaN,NaN
1,2023-01-04,ORCL,84.20,85.17,83.64,84.48,1,1.0091,0.0091,1.0091
2,2023-01-05,ORCL,85.00,85.40,83.22,84.31,1,1.0070,-0.0020,1.0070
3,2023-01-06,ORCL,84.74,86.34,83.82,85.66,1,1.0232,0.0160,1.0232
4,2023-01-09,ORCL,86.84,87.59,86.16,86.42,1,1.0323,0.0089,1.0323


-----------Volume Table-----------


,Date,stock_id,Adj Close,Volume
0,2023-01-03,ORCL,81.88,8997500
1,2023-01-04,ORCL,82.63,7836200
2,2023-01-05,ORCL,82.46,7643800
3,2023-01-06,ORCL,83.78,8641600
4,2023-01-09,ORCL,84.84,7519700


# Create SQLite Database and store the data
## We decided to use EQL Lite because:
- Our code is supposed to run on the local computer and does not need to manage high traffic.
- SQLite can change files into smaller-size archives with lesser metadata.
- SQLite is used as a temporary dataset to get processed with some data within an application.

In [17]:
# Specify the name of the database
db_name = "stock_analysis.db"

# Connecting to sqlite
connect = sqlite3.connect(db_name)

# cursor object 
cursor = connect.cursor()

In [18]:
# Drop the stock table if already exists.
query = """DROP TABLE IF EXISTS stock"""
cursor.execute(query)

# Creating table
table = """
CREATE TABLE stock
(
    Stock_id INT, 
    Stock_name VARCHAR(20) NOT NULL,
    Ticker VARCHAR(10) NOT NULL,
    PRIMARY KEY (Stock_id)
);"""

cursor.execute(table)

In [19]:
query = """
INSERT INTO stock VALUES
    (1, 'Apple', 'AAPL'),
    (2, 'Google', 'GOOGL'),
    (3, 'Intel', 'INTC'),
    (4, 'Microsoft', 'MSTF'),
    (5, 'Oracle', 'ORCL')
"""

cursor.execute(query)

# Commit the changes in the database     
connect.commit() 

In [20]:
# Verify the Stock Table dataset has the desired stock ticker symbols for analysis
data = cursor.execute('''SELECT * FROM stock''')

for row in data: 
    print(row)

(1, 'Apple', 'AAPL')
(2, 'Google', 'GOOGL')
(3, 'Intel', 'INTC')
(4, 'Microsoft', 'MSTF')
(5, 'Oracle', 'ORCL')


# Database Functions

In [21]:
# Create a new function to generate uniform tables
def create_stock_table (table_name):
    query = f'''DROP TABLE IF EXISTS {table_name}'''
    cursor.execute(query)
    table = f'''
    CREATE TABLE {table_name}
    (
        Date TIMESTAMP,
        Stock_id VARCHAR(20),
        Open REAL NOT NULL, 
        High REAL NOT NULL, 
        Low REAL NOT NULL, 
        Close REAL NOT NULL, 
        Month INT NOT NULL,
        Normalization REAL NOT NULL,
        Percentage_change REAL, 
        Accumulation REAL,
        PRIMARY KEY (date, stock_id),
        Constraint fk_stock,
        FOREIGN KEY(Stock_id) REFERENCES stock(Ticker)
    )
    '''   
    cursor.execute(table)
    
    msg = f"The {table_name} table has been created successfully!"
    return msg

In [22]:
# Database function to create volume tables for the given stock ticker symbols
def create_volume_table(table_name):
    query = f'''DROP TABLE IF EXISTS {table_name}'''
    cursor.execute(query)
    table = f"""
    CREATE TABLE {table_name} 
    (
        Date TIMESTAMP,
        Stock_id VARCHAR(20), 
        'Adj Close' REAL NOT NULL,
        Volume INT REAL NOT NULL,
        PRIMARY KEY (date, stock_id),
        Constraint fk_stock,
        FOREIGN KEY(Stock_id) REFERENCES {table_name}(Stock_id)
        FOREIGN KEY(Date) REFERENCES {table_name}(Date)   
    )
    """
    cursor.execute(table) 

    msg = f"The {table_name} table has been created successfully!"
    return msg

In [23]:
def fetching_data(table_name):
    data = cursor.execute(f'''SELECT * FROM {table_name}''')

    for row in data: 
       print(row)

# Creating the tables and Inserting data

In [24]:
create_stock_table("Apple_Price")

'The Apple_Price table has been created successfully!'

In [25]:
apple_df.to_sql("Apple_Price", con = connect, if_exists = "append", index = False)

250

In [26]:
create_volume_table("Apple_Volume")

'The Apple_Volume table has been created successfully!'

In [27]:
apple_volume.to_sql("Apple_Volume", con = connect, if_exists = "append", index = False)

250

# Creating the tables and Inserting data using a for loop

In [28]:
list_data = [
    [google_df, "Google_Price", "Google_Volume"], 
    [microsoft_df, "Microsoft_Price", "Microsoft_Volume"], 
    [intel_df, "Intel_Price", "Intel_Volume"], 
    [oracle_df, "Oracle_Price", "Oracle_Volume"]
]

for item in list_data:
    create_stock_table(item[1])
    item[0][0].to_sql(item[1], con = connect, if_exists = "append", index = False)

    create_volume_table(item[2])
    item[0][1].to_sql(item[2], con = connect, if_exists = "append", index = False)